In [7]:
import requests
import re

cities = ['London,uk', 'Paris,fr', 'Porto,pt']
weather_dict = {}

def city_forecast(city):
  url = "https://community-open-weather-map.p.rapidapi.com/forecast"
  querystring = {"q":city}
  headers = {
    'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
    'x-rapidapi-key': "......"
    }

  response = requests.request("GET", url, headers=headers, params=querystring)
  return response.json()

In [8]:
for city in cities:
  weather_dict[city] = city_forecast(city)

In [9]:
def get_day_weather(pred):
  pattern =re.compile(" [0-9]*:[0-9]*:[0-9]*")
  t = pattern.search(pred['dt_txt'])

  hour = int(t[0].split(':')[0])
  
  if 10<=hour<=19:
        return True

  return False

day_weather = {}

for city in weather_dict.keys():
  day_weather[city] = list(filter(get_day_weather, weather_dict[city]['list']))

In [10]:
def travel_estimator(weather_stat):
  estimation = {}

  for city in weather_stat.keys():    
        estimation[city] = {}
        estimation[city]['clear_sky_count'] = 0
        estimation[city]['av_temp'] = 0

        for prediction in day_weather[city]:
          estimation[city]['av_temp'] += prediction['main']['temp']
          if prediction['weather'][0]['description'] in ['clear sky', 'few clouds']:
            estimation[city]['clear_sky_count'] += 1          

        estimation[city]['av_temp'] = round(estimation[city]['av_temp'] / len(day_weather[city]), 2)
        # convert temperature to celsius
        estimation[city]['av_temp_cels'] = round(estimation[city]['av_temp'] - 273.15, 2)

  return estimation

travel_rating = travel_estimator(day_weather)

# sort cities by clear sky forecasts
sorted_travel_rating = sorted(travel_rating.items() ,  key=lambda x: x[1]['av_temp_cels'] )


for city in sorted_travel_rating:
  print(city[0])
  print('Clear sky forecasts:', city[1]['clear_sky_count'])  
  print('Average temperature:', city[1]['av_temp_cels'], "C")
  print('n')

London,uk
Clear sky forecasts: 2
Average temperature: 14.8 C
n
Paris,fr
Clear sky forecasts: 3
Average temperature: 16.77 C
n
Porto,pt
Clear sky forecasts: 1
Average temperature: 19.13 C
n
